In [1]:
from bs4 import BeautifulSoup
import time
import requests
import pandas as pd
import numpy as np
import re

In [16]:
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'
}

Rank = []
Club = []
Team_ID = []
Matches = []
Wins = []
Draws = []
Losses = []
Goals = []
Goal_Difference = []
Points = []
Season = []
season = 2000
while True:
  # URL for the league table
  base_url = f"https://www.transfermarkt.com/egyptian-premier-league/tabelle/wettbewerb/EGY1?saison_id={season}"
  pageTree = requests.get(base_url, headers=headers)
  soup = BeautifulSoup(pageTree.content, 'html.parser')
  table = soup.find('table', class_='items')

  # Initialize empty lists to store the data

  # Iterate over the rows in the table body
  for row in table.tbody.find_all('tr'):
      # Extract rank
      rank = row.find('td', class_='rechts hauptlink').text.strip()
      Rank.append(rank)

      # Extract club name
      club = row.find('td', class_='no-border-links hauptlink').a.text.strip()
      Club.append(club)
      team_link = row.find('td', class_='no-border-links hauptlink').a.get('href')
      team_id = re.search(r'/(\d+)/', team_link).group(1)
      Team_ID.append(team_id)

      # Extract matches played, wins, draws, losses, goals, points
      stats = row.find_all('td', class_='zentriert')
      Matches.append(stats[1].text.strip())
      Wins.append(stats[2].text.strip())
      Draws.append(stats[3].text.strip())
      Losses.append(stats[4].text.strip())
      Goals.append(stats[5].text.strip())
      Goal_Difference.append(stats[6].text.strip())
      Points.append(stats[7].text.strip())
      Season.append(season)
  if season == 2013:
      table = soup.find('div', {'id':'yw2'})

      for row in table.tbody.find_all('tr'):
          # Extract rank
          rank = row.find('td', class_='rechts hauptlink').text.strip()
          Rank.append(rank)

          # Extract club name
          club = row.find('td', class_='no-border-links hauptlink').a.text.strip()
          Club.append(club)
          team_link = row.find('td', class_='no-border-links hauptlink').a.get('href')
          team_id = re.search(r'/(\d+)/', team_link).group(1)
          Team_ID.append(team_id)

          # Extract matches played, wins, draws, losses, goals, points
          stats = row.find_all('td', class_='zentriert')
          Matches.append(stats[1].text.strip())
          Wins.append(stats[2].text.strip())
          Draws.append(stats[3].text.strip())
          Losses.append(stats[4].text.strip())
          Goals.append(stats[5].text.strip())
          Goal_Difference.append(stats[6].text.strip())
          Points.append(stats[7].text.strip())
          Season.append(season)

  print (season, "Scraped")
  season += 1

  if season > 2024:
        break

2000 Scraped
2001 Scraped
2002 Scraped
2003 Scraped
2004 Scraped
2005 Scraped
2006 Scraped
2007 Scraped
2008 Scraped
2009 Scraped
2010 Scraped
2011 Scraped
2012 Scraped
2013 Scraped
2014 Scraped
2015 Scraped
2016 Scraped
2017 Scraped
2018 Scraped
2019 Scraped
2020 Scraped
2021 Scraped
2022 Scraped
2023 Scraped
2024 Scraped


In [17]:
import csv
from itertools import zip_longest
file_list = [Season, Rank, Club, Points , Wins, Draws, Losses, Goals, Goal_Difference, Team_ID]
exported = zip_longest(*file_list)
with open ("Ranks from 2000 to 2024.csv", "w") as myfile:
  wr = csv.writer (myfile)
  wr.writerow(["Season","Rank", "Club" ,"Points" , "Wins" , "Draws", "Losses", "Goals", "Goals Difference", "Team ID"])
  wr.writerows(exported)

In [18]:
df = pd.read_csv("Ranks from 2000 to 2024.csv")
df.head()

,Season,Rank,Club,Points,Wins,Draws,Losses,Goals,Goals Difference,Team ID
0,2000,1,Zamalek,65,20,5,1,54:18,36,664
1,2000,2,Al Ahly,57,17,6,3,42:17,25,7
2,2000,3,El Masry,46,11,13,2,36:17,19,9094
3,2000,4,Ismaily,43,11,10,5,32:19,13,3595
4,2000,5,Qanah,42,11,9,6,26:18,8,24297


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 414 entries, 0 to 413
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Season            414 non-null    int64 
 1   Rank              414 non-null    int64 
 2   Club              414 non-null    object
 3   Points            414 non-null    int64 
 4   Wins              414 non-null    int64 
 5   Draws             414 non-null    int64 
 6   Losses            414 non-null    int64 
 7   Goals             414 non-null    object
 8   Goals Difference  414 non-null    int64 
 9   Team ID           414 non-null    int64 
dtypes: int64(8), object(2)
memory usage: 32.5+ KB


In [20]:
df[['Scored', 'Get a Goals']] = df['Goals'].str.split(':', expand=True) # Split the 'Goals' column instead of 'Points'.
df.head()

,Season,Rank,Club,Points,Wins,Draws,Losses,Goals,Goals Difference,Team ID,Scored,Get a Goals
0,2000,1,Zamalek,65,20,5,1,54:18,36,664,54,18
1,2000,2,Al Ahly,57,17,6,3,42:17,25,7,42,17
2,2000,3,El Masry,46,11,13,2,36:17,19,9094,36,17
3,2000,4,Ismaily,43,11,10,5,32:19,13,3595,32,19
4,2000,5,Qanah,42,11,9,6,26:18,8,24297,26,18


In [21]:
if 'Rank' in df.columns:
    df['Champion'] = df['Club'].where(df['Rank'] == 1)
elif 'Season' in df.columns:
    df['Champion'] = df['Club'].where(df['Rank'] == 1) & (df['Season'] != 2013)
df.head()

,Season,Rank,Club,Points,Wins,Draws,Losses,Goals,Goals Difference,Team ID,Scored,Get a Goals,Champion
0,2000,1,Zamalek,65,20,5,1,54:18,36,664,54,18,Zamalek
1,2000,2,Al Ahly,57,17,6,3,42:17,25,7,42,17,NaN
2,2000,3,El Masry,46,11,13,2,36:17,19,9094,36,17,NaN
3,2000,4,Ismaily,43,11,10,5,32:19,13,3595,32,19,NaN
4,2000,5,Qanah,42,11,9,6,26:18,8,24297,26,18,NaN


In [24]:
#change index 203 in chapion to null
df.loc[203, 'Champion'] = np.nan
df.head()

,Season,Rank,Club,Points,Wins,Draws,Losses,Goals,Goals Difference,Team ID,Scored,Get a Goals,Champion
0,2000,1,Zamalek,65,20,5,1,54:18,36,664,54,18,Zamalek
1,2000,2,Al Ahly,57,17,6,3,42:17,25,7,42,17,NaN
2,2000,3,El Masry,46,11,13,2,36:17,19,9094,36,17,NaN
3,2000,4,Ismaily,43,11,10,5,32:19,13,3595,32,19,NaN
4,2000,5,Qanah,42,11,9,6,26:18,8,24297,26,18,NaN


In [29]:
df_2013= df[df['Season'] == 2013]
df_2013.head(20)

,Season,Rank,Club,Points,Wins,Draws,Losses,Goals,Goals Difference,Team ID,Scored,Get a Goals,Champion
192,2013,1,Al Ahly,40,12,4,4,30:10,20,7,30,10,Al Ahly
193,2013,2,Smouha,35,10,5,5,24:20,4,23387,24,20,NaN
194,2013,3,Ittihad Alex,31,7,10,3,20:17,3,3963,20,17,NaN
195,2013,4,El Mokawloon,30,7,9,4,27:17,10,3369,27,17,NaN
196,2013,5,El Gouna,26,7,5,8,17:16,1,20572,17,16,NaN
197,2013,6,El Dakhlia,25,5,10,5,18:19,-1,22707,18,19,NaN
198,2013,7,El Makasa,24,5,9,6,17:22,-5,22702,17,22,NaN
199,2013,8,Enppi SC,22,4,10,6,19:19,0,9218,19,19,NaN
200,2013,9,El Raja,21,5,6,9,23:34,-11,28571,23,34,NaN
201,2013,10,El Mahalla,19,3,10,7,13:21,-8,13446,13,21,NaN


In [30]:
df.to_csv("Ranks from 2000 to 2024.csv", index=False)